In [ ]:
import pandas as pd
import numpy as np
import scanpy as sc

import matplotlib.pyplot as plt
import seaborn as sns

import pathlib as pl
import os

In [ ]:
from scipy.stats import mannwhitneyu

In [ ]:
from statsmodels.stats.multitest import multipletests

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
from statannotations.Annotator import Annotator

In [ ]:
def pretty_ax(ax: plt.Axes, linew: int = 1, hideticks: bool=False):
    # Hide the right and top spines
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    for axis in ["bottom", "left"]:
        ax.spines[axis].set_linewidth(linew)
    if hideticks:
        ax.set_xticks([])
        ax.set_yticks([])

# Mitocarta genes

In [ ]:
mitocarta = pd.read_csv("/add/path/here/auxiliary_data/mitocarta_genes.csv").values.ravel()
mitocarta = np.setdiff1d(mitocarta,['MT-ATP6', 'MT-CO2', 'MT-CO1', 'MT-ND2', 'MT-ND4', 'MT-ND5',
       'MT-CYB', 'MT-ATP8', 'MT-CO3', 'MT-ND3', 'MT-ND1', 'MT-ND4L',
       'MT-ND6'])

In [ ]:
mitocarta_pathways = pd.read_csv("/add/path/here/auxiliary_data/mitocarta_pathways.csv",index_col=0)

high_order = []

for name in mitocarta_pathways["MitoPathways Hierarchy"].str.split(">").values:
    if len(name)==1:
        high_order.append(name[0])

high_order = mitocarta_pathways[mitocarta_pathways["MitoPathways Hierarchy"].isin(high_order)]["Genes"].str.split(", ").to_dict()

In [ ]:
high_order.keys()

# Analyze mitocarta pathways 

In [ ]:
resdir = pl.Path("/add/path/here/metabolic_dysregulation/mitocarta_met_res")

In [ ]:
mitocarta_enrichment = {}
for f in resdir.iterdir():
    if f.stem in [".DS_Store"]:
        continue
    mitocarta_enrichment[f.stem] = pd.read_csv(f, index_col=0)

In [ ]:
meta_pathways = mitocarta_pathways[mitocarta_pathways["MitoPathways Hierarchy"].str.contains("Metabolism")].index

In [ ]:
meta_mito = []
for ct in mitocarta_enrichment:
    print(ct)
    meta_ct = mitocarta_enrichment[ct].loc[meta_pathways]
    meta_ct = meta_ct["Diff. median"]
    meta_ct.name = ct
    meta_mito.append(meta_ct)

In [ ]:
meta_mito = pd.concat(meta_mito,axis=1).rename(columns={"Pancreas_Raghavan_10X": "MetPancreas", "SCLC_Chan_10X": "SCLC",
                                           "RCC_Bi_10X": "RCC", "LUAD_Bischoff_10X": "LUAD", 
                                            "Breast_Wu_10X": "Breast", 
                                                        "Pancreas_Steele_10X": "Pancreas", 
                                                        "UvealMelanoma_Durante_10X": "Uveal Melanoma",})

In [ ]:
(meta_mito>0).sum(axis=1).sort_values(ascending=False).head(10)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(4,6))
sns.heatmap(meta_mito.loc[meta_mito.median(axis=1).sort_values(ascending=False).index,
            ["Uveal Melanoma","LUAD","SCLC","RCC","Breast","Pancreas","MetPancreas"]],
            cmap="vlag", center=0, vmin=-0.1, vmax=0.1,ax=ax)
ax.set_yticks([])
ax.set_ylabel("")
fig.savefig("/add/path/here/figures/mitocarta_meta_dys.svg", 
                 dpi=200, bbox_inches='tight')

In [ ]:
(meta_mito>0).sum(axis=1).sort_values(ascending=False).head(10)

In [ ]:
meta_mito.loc[meta_mito.median(axis=1).sort_values(ascending=False).index].head(15)